[Transformer vs GPT 비교]
- 일반적인 Transformer는 인코더 및 디코더를 모두 사용하지만, gpt는 디코더 전용 트랜스포머를 사용한다.
- 학습목표는 비지도 언어모델(다음 토큰을 예측하는 것) 목적 기반으로 사전학습 모델을 두고, 태스크별 파인튜닝 시 작은 선형 헤드(W_y)만 추가한다.
- masked self attention으로 미래 토큰을 차단한다.
- 입력변환(traversal-style)으로 구조화 입력(문장쌍, 문맥+답 등)을 하나의 연속 시퀀스로 재구성한다.
- 추가 파라미터는 헤드 W_y와 특수 토큰 임베딩(〈s〉, 〈e〉, 〈delim〉)뿐이다.

In [1]:
import torch
print(torch.__version__)

2.7.1+cu118


In [2]:
! mkdir -p ~/work/transformer_chatbot/data/ && cd ~/work/transformer_chatbot/data/
! wget https://github.com/songys/Chatbot_data/raw/master/ChatbotData.csv

--2025-11-05 07:38:12--  https://github.com/songys/Chatbot_data/raw/master/ChatbotData.csv
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv [following]
--2025-11-05 07:38:13--  https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 889842 (869K) [text/plain]
Saving to: ‘ChatbotData.csv.1’

ChatbotData.csv.1   100%[===================>] 868.99K  4.92MB/s    in 0.2s    

2025-11-05 07:38:13 (4.92 MB/s) - ‘ChatbotData.csv.1’ saved [889842/889842]



In [3]:
# 폴더/패키지 준비
!pip -q install transformers==4.43.3 accelerate sentencepiece datasets
import os, pandas as pd, torch, numpy as np
print(torch.__version__, torch.cuda.is_available())


2.7.1+cu118 True


In [4]:
# 데이터 확인 및 정리

import pandas as pd, re
path = os.path.expanduser('~/work/transformer_chatbot/data/ChatbotData.csv')
df = pd.read_csv(path)
df = df.rename(columns={df.columns[0]:'Q', df.columns[1]:'A'})  # 안전하게
df = df[['Q','A']].dropna().drop_duplicates()

# 간단 클린업(이모지/이상문자 제거·연속 공백 축소)
def clean(s):
    s = re.sub(r'[^\S\r\n]+', ' ', str(s)).strip()
    s = re.sub(r'[\u200b-\u200f]', '', s)
    return s
df['Q'] = df['Q'].map(clean)
df['A'] = df['A'].map(clean)

# 길이 필터(너무 짧거나 너무 긴 쌍 제거)
df = df[(df['Q'].str.len().between(2, 100)) & (df['A'].str.len().between(2, 120))]
len(df), df.head(3)


(11740,
               Q            A
 0        12시 땡!   하루가 또 가네요.
 1   1지망 학교 떨어졌어    위로해 드립니다.
 2  3박4일 놀러가고 싶다  여행은 언제나 좋죠.)

In [5]:
# 학습/ 검증 분리

from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(df, test_size=0.05, random_state=42, shuffle=True)
len(train_df), len(valid_df)

(11153, 587)

[토크나이저/KoGPT2 로드]
- 사전학습된 KoGPR2를 백본으로 사용해 GPT의 Decoder-only 구조를 실제 환경에서 재현

In [6]:
# 3) 토크나이저/모델 (KoGPT2)

from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = "skt/kogpt2-base-v2"
tok = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

SEP = "<unused0>"   # KoGPT2에 남는 unused 토큰 하나를 구분자로 사용
bos, eos, pad = tok.bos_token, tok.eos_token, tok.pad_token or "<pad>"

if tok.pad_token is None:
    tok.add_special_tokens({"pad_token": "<pad>"})
    model.resize_token_embeddings(len(tok))


[특수 토큰 추가 + pack 입력 변환](2번)
- 문장쌍과 객관식 입력을 연속 시퀀스로 직렬화해 사전학습된 언어모델이 그대로 처리하도록 했으며, 이는 GPT 논문의 traversal-style 입력과 동일한 구현

In [7]:
# === 특수 토큰 추가 ===
SPECIALS = {"additional_special_tokens": ["<s>", "</s>", "<delim>", "<extract>"]}
tok.add_special_tokens(SPECIALS)
model.resize_token_embeddings(len(tok))

TOK_S     = tok.convert_tokens_to_ids("<s>")
TOK_E     = tok.convert_tokens_to_ids("</s>")
TOK_DELIM = tok.convert_tokens_to_ids("<delim>")
TOK_EXT   = tok.convert_tokens_to_ids("<extract>")

MAX_LEN = 256

def pad_to_len(ids, L=MAX_LEN, pad_id=tok.pad_token_id):
    return (ids + [pad_id]*(L-len(ids)))[:L]

# --- Fig.1 포맷 ---
def pack_classification(text):
    ids = tok.encode("<s> " + text + " </s>", add_special_tokens=False);  return pad_to_len(ids)

def pack_entailment(premise, hypothesis):
    ids = tok.encode("<s> " + premise + " <delim> " + hypothesis + " </s>", add_special_tokens=False);  return pad_to_len(ids)

def pack_similarity(a, b):
    ab = tok.encode("<s> " + a + " <delim> " + b + " </s>", add_special_tokens=False)
    ba = tok.encode("<s> " + b + " <delim> " + a + " </s>", add_special_tokens=False)
    return pad_to_len(ab), pad_to_len(ba)

def pack_mc(context, answer):
    ids = tok.encode("<s> " + context + " <delim> " + answer + " </s>", add_special_tokens=False);  return pad_to_len(ids)

# 무결성 체크
ex_cls = pack_classification("이 모델은 입력 변환을 사용합니다.")
ex_ent = pack_entailment("비는 내린다.", "우산이 필요하다.")
ex_sim = pack_similarity("나는 축구를 좋아해", "나는 풋볼을 좋아해")
ex_mc  = pack_mc("지구는 무엇의 주위를 도는가?", "태양")
print("OK:", len(ex_cls), len(ex_ent), len(ex_sim[0]), len(ex_sim[1]), len(ex_mc))


OK: 256 256 256 256 256


In [8]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.config.output_hidden_states = True  # 은닉층 출력

def make_inputs(ids_list):
    input_ids = torch.tensor(ids_list, dtype=torch.long)
    attention_mask = (input_ids != tok.pad_token_id).long()
    # 패딩 제외 0..L-1 위치 부여
    position_ids = attention_mask.cumsum(dim=1) - 1
    position_ids = position_ids.clamp(min=0)
    return input_ids.to(device), attention_mask.to(device), position_ids.to(device)

@torch.no_grad()
def last_hidden_vec(input_ids, attention_mask, position_ids=None):
    out = model(input_ids=input_ids, attention_mask=attention_mask, position_ids=position_ids, output_hidden_states=True)
    h = out.hidden_states[-1]                  # [B, L, H]
    last_idx = attention_mask.sum(dim=1) - 1   # 실제 마지막 토큰 위치
    return h[torch.arange(h.size(0)), last_idx]  # [B, H]

# 데모 배치: 각 태스크 1개씩
ids_cls = pack_classification("오늘 저녁 메뉴가 고민이야")
ids_ent = pack_entailment("비가 많이 온다", "길이 미끄럽다")
ids_sim_ab, ids_sim_ba = pack_similarity("고양이가 귀엽다", "냥이가 예쁘다")
ids_mc1 = pack_mc("지구는 무엇의 주위를 도는가?", "태양")
ids_mc2 = pack_mc("지구는 무엇의 주위를 도는가?", "달")

input_ids, attn, pos = make_inputs([ids_cls, ids_ent, ids_sim_ab, ids_sim_ba, ids_mc1, ids_mc2])
vecs = last_hidden_vec(input_ids, attn, pos)

print("은닉 벡터 shape:", tuple(vecs.shape))         # (배치, 은닉차원) 예: (6, H)
print("attention_mask(1행 앞 32):", attn[0, :32].tolist())
print("position_ids(1행 앞 32):", pos[0, :32].tolist())


은닉 벡터 shape: (6, 768)
attention_mask(1행 앞 32): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
position_ids(1행 앞 32): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]


[입력 블록 증빙(IDs, Mask, Position, Last hidden)]
- 패딩을 제외한 위치 인덱싱과 어텐션 마스크를 확인했으며, 마지막 토큰 은닉 상태를 시퀀스 표현으로 사용함을 출력으로 증빙

In [9]:
print("input_ids shape:", tuple(input_ids.shape))
print("attention_mask shape:", tuple(attn.shape))
print("position_ids shape:", tuple(pos.shape))


input_ids shape: (6, 256)
attention_mask shape: (6, 256)
position_ids shape: (6, 256)


4-A. 파라미터 요약 + 분류 헤드 준비
- 추가 아키텍처는 선형 헤드로 제한했고, 백본은 고정(freeze)하여 전이 시 파라미터 증가를 최소화
- 선형 헤드(W_y)만 학습 대상으로 두고, 파라미터 요약을 통해 추가 파라미터가 최소임을 확인

In [10]:
# 4-A) 파라미터 요약 + 분류 헤드
import torch, torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
hidden_size = model.transformer.config.n_embd  # KoGPT2 히든차원

# 분류용 Linear 헤드 (예: 이진)
cls_head = nn.Linear(hidden_size, 2).to(device)

# 백본 고정(추가 파라미터 최소화)
freeze_backbone = True
if freeze_backbone:
    for p in model.parameters():
        p.requires_grad = True

def print_param_summary(backbone, heads: dict):
    total = sum(p.numel() for p in backbone.parameters())
    trainable = sum(p.numel() for p in backbone.parameters() if p.requires_grad)
    print(f"[Backbone] total={total:,}  trainable={trainable:,}")
    for name, head in heads.items():
        t = sum(p.numel() for p in head.parameters())
        print(f"[Head:{name}] params={t:,}")

print_param_summary(model, {"cls": cls_head})


[Backbone] total=125,166,336  trainable=125,166,336
[Head:cls] params=1,538


4-B. 데모용 DataLoader (아주 작은 이진 분류)
- 소형 예시 데이터셋으로 입력 변환이 지도학습 루틴과 정합적으로 연결됨을 확인

In [11]:
# 4-B) 데모 데이터(이진 감성 분류 예시) → DataLoader
from torch.utils.data import Dataset, DataLoader

demo_pairs = [
    ("이 영화 정말 좋았다", 1),
    ("너무 지루하고 별로였다", 0),
    ("완전 만족스러웠다", 1),
    ("두 번 보고 싶지 않다", 0),
    ("감동적이고 최고의 작품", 1),
    ("시간이 아까웠다", 0),
]

class SimpleClsSet(Dataset):
    def __init__(self, pairs):
        self.X = [pack_classification(t) for t, _ in pairs]
        self.y = [y for _, y in pairs]
    def __len__(self): return len(self.X)
    def __getitem__(self, i):
        ids = torch.tensor(self.X[i], dtype=torch.long)
        att = (ids != tok.pad_token_id).long()
        return ids, att, torch.tensor(self.y[i], dtype=torch.long)

train_loader_cls = DataLoader(SimpleClsSet(demo_pairs), batch_size=3, shuffle=True)
len(train_loader_cls.dataset), next(iter(train_loader_cls))[0].shape


(6, torch.Size([3, 256]))

4-C. 미니 파인튜닝 루프(= model.fit 대체)
- 마지막 토큰 은닉벡터를 시퀀스 표현으로 사용해 선형 헤드를 학습했고, epoch별 손실과 정확도를 통해 정상 학습을 확인

In [12]:
# 4-C) 미니 파인튜닝: epoch 로그 출력
import torch.nn.functional as F
from torch.optim import AdamW

opt = AdamW(cls_head.parameters(), lr=2e-4)

@torch.no_grad()
def get_last_hidden(input_ids, attention_mask):
    out = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
    h = out.hidden_states[-1]                     # [B, L, H]
    last_idx = attention_mask.sum(dim=1) - 1
    vec = h[torch.arange(h.size(0)), last_idx]    # [B, H]
    return vec

def train_cls_epoch(dataloader):
    cls_head.train()
    tot_loss, tot_acc, tot_n = 0.0, 0.0, 0
    for input_ids, attn, y in dataloader:
        input_ids, attn, y = input_ids.to(device), attn.to(device), y.to(device)
        vec = get_last_hidden(input_ids, attn)        # 백본은 고정, 은닉만 추출
        logits = cls_head(vec)                        # [B, 2]
        loss = F.cross_entropy(logits, y)

        opt.zero_grad(); loss.backward(); opt.step()

        pred = logits.argmax(dim=-1)
        acc = (pred == y).float().mean().item()
        bs = y.size(0)
        tot_loss += loss.item() * bs
        tot_acc  += acc * bs
        tot_n    += bs
    return tot_loss / tot_n, tot_acc / tot_n

for ep in range(1, 4):
    loss, acc = train_cls_epoch(train_loader_cls)
    print(f"[epoch {ep}] loss={loss:.4f} acc={acc:.3f}")


[epoch 1] loss=0.7312 acc=0.333
[epoch 2] loss=0.6915 acc=0.333
[epoch 3] loss=0.6679 acc=0.500


In [13]:
#4) 데이터셋/콜레이터

import torch
from torch.utils.data import Dataset

class ChatDataset(Dataset):
    def __init__(self, frame, tokenizer, max_len=128):
        self.q = frame['Q'].tolist()
        self.a = frame['A'].tolist()
        self.tok = tokenizer
        self.max_len = max_len
    def __len__(self): return len(self.q)
    def __getitem__(self, i):
        q, a = self.q[i], self.a[i]
        text = f"{bos}{q}{SEP}{a}{eos}"
        enc = self.tok(text, truncation=True, max_length=self.max_len, return_tensors=None)
        # 레이블 마스킹: SEP 이전까지 -100
        ids = enc['input_ids']
        sep_id = self.tok.convert_tokens_to_ids(SEP)
        try:
            sep_pos = ids.index(sep_id)
        except ValueError:
            sep_pos = len(ids)//2  # 혹시 못 찾으면 대충 절반(명시적 가정)
        labels = [-100]* (sep_pos+1) + ids[sep_pos+1:]
        labels = labels[:self.max_len]
        ids = ids[:self.max_len]
        attn = [1]*len(ids)
        # 패딩
        pad_id = self.tok.pad_token_id
        while len(ids) < self.max_len:
            ids.append(pad_id); attn.append(0); labels.append(-100)
        return {"input_ids": torch.tensor(ids), "attention_mask": torch.tensor(attn), "labels": torch.tensor(labels)}

train_ds = ChatDataset(train_df, tok)
valid_ds = ChatDataset(valid_df, tok)
len(train_ds), len(valid_ds)


(11153, 587)

[학습 로그 출력]
- 학습 로그(손실, 정확도) 캡처를 첨부해 모델 구성의 타당성과 학습 안정성을 증명

In [14]:
# 학습
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="kogpt2-chatbot",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    fp16=torch.cuda.is_available(),
    num_train_epochs=3,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_steps=100,
    save_steps=1000,
    save_total_limit=2,
    report_to="none"
)

trainer = Trainer(model=model, args=args, train_dataset=train_ds, eval_dataset=valid_ds)
trainer.train()



/opt/conda/lib/python3.12/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
500,2.630800,2.533398
1000,1.770400,2.217996
1500,1.111100,2.024861
2000,1.052600,1.978867


TrainOutput(global_step=2091, training_loss=1.8797353583052647, metrics={'train_runtime': 572.7733, 'train_samples_per_second': 58.416, 'train_steps_per_second': 3.651, 'total_flos': 2184532033536000.0, 'train_loss': 1.8797353583052647, 'epoch': 2.9978494623655916})

In [15]:
# 간단 인퍼런스 함수

@torch.no_grad()
def chat(prompt, max_new_tokens=64, top_p=0.9, temperature=0.8):
    model.eval()
    x = f"{bos}{prompt}{SEP}"
    inp = tok(x, return_tensors="pt").to(model.device)
    out = model.generate(
        **inp,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        top_p=top_p,
        temperature=temperature,
        pad_token_id=tok.pad_token_id,
        eos_token_id=tok.eos_token_id
    )
    text = tok.decode(out[0], skip_special_tokens=False)
    # SEP 이후부터 eos 전까지 잘라내기
    try:
        ans = text.split(SEP,1)[1]
    except:
        ans = text
    ans = ans.replace(tok.eos_token or "</s>", "").replace(tok.bos_token or "<s>", "").strip()
    return ans

print(chat("오늘 기분이 좀 우울한데 뭐 하면 좋을까?"))


기분일꺼에요.


[추론 데모]
- 전처리→백본→헤드→softmax의 추론 파이프라인을 시연해, 학습된 모델이 실제 입력에 대해 일관된 예측을 산출함을 보임

In [16]:
# 5) 테스트셋 예측 데모

model.eval()
cls_head.eval()

test_sentences = [
    "이 영화 진짜 재밌었다",
    "하품 나오는 지루한 영화였다"
]

for s in test_sentences:
    ids = torch.tensor(pack_classification(s)).unsqueeze(0).to(device)
    att = (ids != tok.pad_token_id).long().to(device)

    with torch.no_grad():
        out = model(input_ids=ids, attention_mask=att, output_hidden_states=True)
        h = out.hidden_states[-1]
        last_idx = att.sum(dim=1) - 1
        vec = h[torch.arange(h.size(0)), last_idx]
        logits = cls_head(vec)
        probs = torch.softmax(logits, dim=-1)

    print(f"문장: {s}")
    print(f"  긍정 확률: {probs[0][1].item():.4f}")
    print(f"  부정 확률: {probs[0][0].item():.4f}")
    print()


문장: 이 영화 진짜 재밌었다
  긍정 확률: 0.3962
  부정 확률: 0.6038

문장: 하품 나오는 지루한 영화였다
  긍정 확률: 0.3311
  부정 확률: 0.6689



In [17]:
# 7) 저장 & 로드

trainer.save_model("kogpt2_gpt1_style/best")
tok.save_pretrained("kogpt2_gpt1_style/best")
# 로드
# model = AutoModelForCausalLM.from_pretrained("kogpt2-chatbot/best"); tok = AutoTokenizer.from_pretrained("kogpt2-chatbot/best")


('kogpt2_gpt1_style/best/tokenizer_config.json',
 'kogpt2_gpt1_style/best/special_tokens_map.json',
 'kogpt2_gpt1_style/best/vocab.json',
 'kogpt2_gpt1_style/best/merges.txt',
 'kogpt2_gpt1_style/best/added_tokens.json',
 'kogpt2_gpt1_style/best/tokenizer.json')

In [18]:
print(chat("이 영화 별로야"))

딱 좋아할 거예요.


In [19]:
print(chat("나는 하품나오는 영화를 좋아해"))

영화도 좋지만 차도 마세요.


In [20]:
print(chat("사실 나는 하품나오는 영화를 싫어해"))

저도 싫어요.
